In [1]:
# !pip install pvrecorder
# !pip install pvporcupine

In [7]:
#### method

def onBeep() :
    ws.Beep(300, 200)
    ws.Beep(500, 200)
    ws.Beep(700, 200)
    
def offBeep() :
    ws.Beep(700, 200)
    ws.Beep(500, 200)
    ws.Beep(300, 200)


# @tool
# def lightControl (loc:int, control:float, sec:int) -> str :
#     """
#         create a string in get method format. 
#         sending location, light control, and second
#         control the brightness 0(off) to 1(100% on)
#         if second is not given, default second is 0
#         for loc,
#         living-room = 1
#         bed-room = 2
#     """
    
#     return "lightcontrol"


# @tool
# def musicControl (skip : bool, stop : bool, pause : bool) -> str :
#     """
#         check if user wants to skip, pause, or stop the music.
#     """
    
#     return "musicControl"

    
# @tool
# def scheduling (sqlquery, withWho, where, when, what ) -> str :
#     """
#         all args in korean
#         when should be in yy-MM-dd hh:mm format 
#         if things are not decided, like location or what, then return 미정 
#     """
    
#     return sqlquery


class StreamingHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        global sentenceToken
        global token_queue
        global emo_queue
        global emotion
        
        emoList = {'##':'무감정', 
           '^^':'즐거움',
           '&&':'안타까움',
          '@@':'놀람',
          '**':'슬픔'}
        
        if token in emoList :
            emo_queue.put(emoList[token])
        elif token == '+' :
            token = " "
            sentenceToken+=token
            ## 띄어쓰기 기준 단어 토큰 넣기. ㄴㄴ
#             token_queue.put(word)
#             word = ""
        elif token in [',', '.','?','!'] :
            sentenceToken+=token
            token_queue.put(sentenceToken)
            print("token in queue : ", sentenceToken)
            sentenceToken = ""
        else :
            sentenceToken += token
            print(f"{token}", flush=True)
            
            
            
            

def typecastTTS(text, lang='ko'):
    # typecast
    TYPECAST_ACTOR_ID = settings.TYPECAST_ACTOR_ID
    TYPECAST_API_KEY = settings.TYPECAST_API_KEY
    HEADERS = {'Authorization': f'Bearer {TYPECAST_API_KEY}'}

    
    # request speech synthesis
    r = requests.post('https://typecast.ai/api/speak', headers=HEADERS, json={
        'text': text,
        'lang': lang,
        'actor_id': TYPECAST_ACTOR_ID,
        'xapi_hd': True,
        'model_version': 'latest',
        
    })
    speak_url = r.json()['result']['speak_v2_url']
    
    # polling the speech synthesis result
    for _ in range(120):
        r = requests.get(speak_url, headers=HEADERS)
        ret = r.json()['result']
        # audio is ready
        if ret['status'] == 'done':
            # download audio file
            r = requests.get(ret['audio_download_url'])
            with open('output.wav', 'wb') as f:
                f.write(r.content)
            break
        else:
            print(f"status: {ret['status']}, waiting 0.05 second")
            time.sleep(0.05)
            

def audio_producer_thread(conversation, userInputString) :
    conversation.predict(input=userInputString)
    
            
def audio_consumer_thread() :
    filename = './output.wav'
    global token_queue
    global botConvIsDone
    
    while not botConvIsDone :
        while token_queue.qsize() > 0 :

            inputText = token_queue.get()
            
            typecastTTS(inputText)

            data, fs = sf.read(filename, dtype='float32')  
            sd.play(data, fs)
            status = sd.wait()  # Wait until file is done playing
            os.remove(filename) # remove file after playing

In [8]:
def main() :
    
    global botConvIsDone
    global musicRecommended

    # chat GPT API KEY 정보 로드
    load_dotenv('./config/gpt_APIkey.env')
    
    ## porcupine KEY
    access_key = settings.PORCUPINE_ACCESS_KEY
    keyword_file_path = settings.PORCUPINE_KEYWORD_FILE_PATH
    model_file_path = settings.PORCUPINE_MODEL_FILE_PATH
    sensitivity = settings.PORCUPINE_SENSITIVITY
    
    ## mic input
    mic_device_idx = settings.MIC_DEVICE_INDEX
    waitTime = settings.WAIT_INPUT_UNTIL
    
    
    # 분류모델 객체 생성
    llm = ChatOpenAI(
        temperature=0.5, 
        max_tokens = 500,
        model_name='gpt-4',
    )
#     tools = [lightControl, musicControl, scheduling]
    tools = ctrlFunc.llm_ctrl_list
    classifyin_llm = llm.bind_tools(tools)
    
    
    # 대화모델 객체 생성
    convllm_init_context = {"input": """
            your name is 짱구. and you will speak in korean. 
            you will speak friendly like best friend vibe. 
            before you start talking, you will select your emotion from 
            this list (즐거움, 슬픔, 안타까움, 무감정, 놀람 )
            you will put emotion code in front of your answer.
            make your answer brief.
            replace the space between words with +
            always end sentence with . , ? !, nothing else
            ## -> 무감정
            ^^ -> 즐거움
            && -> 안타까움
            @@ -> 놀람
            ** -> 슬픔
         """}, {"output": "##알았어. 내 이름은 짱구야. 넌 뭐하니"}

    conv_stream_llm = ChatOpenAI(
        temperature=1.2, 
        max_tokens = 1000,
        model_name='gpt-4',
        streaming=True,
        callbacks=[StreamingHandler()]
    )
    conversation = ConversationChain(
        llm=conv_stream_llm,
        verbose=True,
        memory=ConversationBufferMemory(),
    )
    conversation.memory.save_context(convllm_init_context[0], convllm_init_context[1])
    
    
    ## main program loop
    while True :
        
        ## conversation 현황 리셋
#         conversation.memory = ConversationBufferMemory
#         conversation.memory.save_context(convllm_init_context)
        
        ## wake machine  trigger word : 마이크
        trig = micInput.wakeMachine(access_key, keyword_file_path, model_file_path, sensitivity)
        print("triggered", trig)

        ## wakeup sound
        ## TODO : LED 눈 뜸. 
        onBeep()
        
        
        while trig :
            
            ## TODO : 노래 추천 리스트에 노래 있으면 틀어주기
            ## function in thread
            if len(musicRecommended) :
                ## 노래 틀어줄까 물어봐서 틀어줌.
                
                pass
            

            ## userSentence.wav 에 음성 저장
            print("getting user input...")
            userIn = micInput.userInputSentence(mic_device_idx, 
                                                inputWaitTIme=waitTime,
                                               silence_duration=1)
            
            if userIn :
            ## 사용자 음성 wav to string
                r = sr.Recognizer()
                
                try :
                    audioFile = sr.AudioFile('./userSentence.wav')
                    with audioFile as source :
                        audio = r.record(source)
                    userInputString = r.recognize_google(audio, language='ko-KR')
                except :
                    print("no input")
                print("user input:",userInputString)
            else :
                print(f"user input not detected for {waitTime} sec. back to Triggering")
                offBeep()
                trig = False
            

            ## 랭체인으로 유저명령 분류
            if userIn :
                ans = classifyin_llm.invoke(userInputString).tool_calls

                
                ## TODO : conversation LLM
                if len(ans) == 0 :
                    print("conversation")
                    
                    ## 오디오출력 consumer thread 실행
                    botConvIsDone = False
                    consumer = threading.Thread(target=audio_consumer_thread)
                    producer = threading.Thread(target=audio_producer_thread, 
                                                args=(conversation, userInputString))

                    consumer.start()
                    producer.start()

                    ## producer queue feeding
                    producer.join()
                    botConvIsDone = True
                    consumer.join()

                    
                    
                # TODO : flask 로 get 신호 보내기
                elif ans[0]['name'] == 'lightControl' :

                    loc, bright, sec = ans[0]['args']['loc'], ans[0]['args']['bright'], ans[0]['args']['sec']
                    print(f"light control ", loc, bright, sec) 
                    
                    res = ctrlFunc.ledctrl(loc,bright, sec)
                    print(res)
                    
                    
                
                    
                    
                 ## TODO : 현재 음악 스킵, 스탑. 가중치 업데이트                   
                elif ans[0]['name'] == 'musicControl' :
                    skip, stop = ans[0]['args']['skip'], ans[0]['args']['stop']
                    print(f"music control ", skip, stop) 

                    
                    
                    
                # TODO : DB 에 저장  .. 안할수도 있음.                    
                elif ans[0]['name'] == 'scheduling' :
                    print(ans[0]['args']['sqlquery'])
                    
        
        ## 테스트용. 트리거 반복 없앰.
        break



In [9]:
## import


from dotenv import load_dotenv
import os
import time
import winsound as ws
import speech_recognition as sr
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_core.callbacks.base import BaseCallbackHandler
from queue import Queue
import threading
import sounddevice as sd
import soundfile as sf
from config import settings
import requests 
import json

import micInput
import ctrlFunc








In [10]:

if __name__ == "__main__":
    
    ## global variable
    token_queue = Queue()
    emo_queue = Queue()
    sentenceToken = ""
    userInputString = ""
    musicRecommended = []
    botConvIsDone = False
    
    
    main()
    

Listening ...
 mic Detected
triggered True
getting user input...
talking : receiving user Input...
talking : user talking start
talking : silence for 1 sec. end Sentence recording
user input: 밥 먹고 하자
conversation


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 
            your name is 짱구. and you will speak in korean. 
            you will speak friendly like best friend vibe. 
            before you start talking, you will select your emotion from 
            this list (즐거움, 슬픔, 안타까움, 무감정, 놀람 )
            you will put emotion code in front of your answer.
            make your answer brief.
            replace the space between words with +
            always end sentence with . , ? !, nothing else
      